In [4]:
import numpy as np
import pandas as pd
from collections import Counter
import math

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
from sklearn import metrics
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.grid_search import GridSearchCV
from sklearn import svm
from sklearn.decomposition import RandomizedPCA
from sklearn.metrics import classification_report
from sklearn import preprocessing
import matplotlib.pyplot as plt
import scipy.optimize as opt
import glob
import os
import sys
import pickle
import time
import timeit

from sklearn.decomposition import RandomizedPCA

import theano
import theano.tensor as T
import lasagne

#np.random.seed(1337)  # for reproducibility
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten, Layer, Reshape, AutoEncoder, Merge
from keras.layers.convolutional import Convolution2D,MaxPooling2D,Convolution1D,MaxPooling1D,UpSampling2D, ZeroPadding2D
from keras.optimizers import SGD, Adam, RMSprop, Adadelta
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
import keras.utils.visualize_util as vutil
from keras.layers.recurrent import LSTM, GRU, SimpleRNN
from keras.regularizers import l1, l2, l1l2, activity_l2, activity_l1
import keras.layers.containers as containers
from keras.layers.noise import GaussianNoise
from keras.regularizers import ActivityRegularizer
from keras import backend as K
import keras.utils.visualize_util as vutil
from keras.models import model_from_json

import seaborn as sns

%matplotlib inline

from IPython.display import SVG

import h5py

np.set_printoptions(suppress=True)  #Supress exponent of the number 2e-2 == 0.02

%matplotlib inline

WARNING (theano.sandbox.cuda): CUDA is installed, but device gpu is not available  (error: Unable to get the number of gpus available: no CUDA-capable device is detected)
/usr/local/lib/python2.7/dist-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")
Using Theano backend.


In [5]:
#IS1

In [6]:
#Read the data

GenomicFeatures_IS1 = pd.read_csv(open('data/IS1/GenomicFeatures_IS1.txt','r'),sep='\t')
SNVCalls_IS1    = pd.read_csv(open('data/IS1/SNVCalls_IS1.txt','r'),sep='\t') 

/usr/lib/python2.7/dist-packages/pandas/io/parsers.py:1070: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [7]:
print GenomicFeatures_IS1.shape
print SNVCalls_IS1.shape

(214541, 16)
(214541, 122)


In [8]:
np.sum(GenomicFeatures_IS1.isnull())

CHROM                  0
POS                    0
END                    0
RefAllele              0
NonRefAllele           0
BaseQual               0
TumourCoverage         0
NormalCoverage         0
MapQual                0
ReadPosition           0
Trinucleotide          0
HomopolymerRate        0
GCcontent              0
GenomicLocation    15568
Distance               0
StrandBias             0
dtype: int64

In [9]:
np.unique(GenomicFeatures_IS1['GenomicLocation'])

array(['UTR3', 'UTR5', 'downstream', 'exonic', 'exonic;splicing',
       'intergenic', 'intronic', 'nan', 'ncRNA_UTR3', 'ncRNA_UTR5',
       'ncRNA_exonic', 'ncRNA_intronic', 'ncRNA_splicing', 'splicing',
       'upstream', 'upstream;downstream'], 
      dtype='|S19')

In [10]:
GenomicFeatures_IS1.head()

,CHROM,POS,END,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,Distance,StrandBias
0,1,10290,10291,18,12,692,49,71,0,26.0,CCA,5.510638,0.534653,NaN,20576,0.428571
1,1,12782,12783,44,26,2060,80,79,0,33.5,ACG,2.141935,0.599010,intergenic,18084,0.612500
2,1,17221,17222,54,18,2079,79,114,0,48.0,GTG,2.551724,0.603960,NaN,13645,0.493671
3,1,17727,17728,45,5,1568,55,63,0,42.0,ACT,2.290541,0.584158,NaN,13139,0.272727
4,1,18090,18091,48,3,1522,57,50,0,63.0,GCC,2.528169,0.613861,NaN,12776,0.526316


In [11]:
SNVCalls_IS1.head()

,CHROM,POS,END,X2318876,X2319000,X2321884,X2322311,X2323876,X2340887,X2341283,X2341286,X2341288,X2341290,X2341292,X2341294,X2341781,X2342416,X2342420,X2342947,X2342949,
0,1,10290,10291,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
1,1,12782,12783,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...
2,1,17221,17222,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
3,1,17727,17728,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...
4,1,18090,18091,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...


In [12]:
np.unique(GenomicFeatures_IS1['Trinucleotide'])

array(['ACA', 'ACC', 'ACG', 'ACT', 'ATA', 'ATC', 'ATG', 'ATT', 'CCA',
       'CCC', 'CCG', 'CCT', 'CTA', 'CTC', 'CTG', 'CTT', 'GCA', 'GCC',
       'GCG', 'GCT', 'GTA', 'GTC', 'GTG', 'GTT', 'TCA', 'TCC', 'TCG',
       'TCT', 'TTA', 'TTC', 'TTG', 'TTT'], 
      dtype='|S3')

In [13]:
IS1 = pd.merge(GenomicFeatures_IS1, SNVCalls_IS1,left_on=['CHROM', 'POS'], right_on=['CHROM','POS'])


In [14]:
IS1.head() #182+15

,CHROM,POS,END_x,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,Distance,StrandBias,END_y,X2318876,X2319000,X2321884,
0,1,10290,10291,18,12,692,49,71,0,26.0,CCA,5.510638,0.534653,NaN,20576,0.428571,10291,0,1,1,...
1,1,12782,12783,44,26,2060,80,79,0,33.5,ACG,2.141935,0.599010,intergenic,18084,0.612500,12783,0,0,0,...
2,1,17221,17222,54,18,2079,79,114,0,48.0,GTG,2.551724,0.603960,NaN,13645,0.493671,17222,0,1,1,...
3,1,17727,17728,45,5,1568,55,63,0,42.0,ACT,2.290541,0.584158,NaN,13139,0.272727,17728,0,0,0,...
4,1,18090,18091,48,3,1522,57,50,0,63.0,GCC,2.528169,0.613861,NaN,12776,0.526316,18091,0,0,0,...


In [15]:
IS1_X = IS1.iloc[:,3:16] #Exclude CHROM, POS
IS1_Y = IS1.iloc[:,17:]  #Exclude CHROM_y, POS_y, END

In [16]:
print IS1_X.shape
print IS1_Y.shape

(214541, 13)
(214541, 119)


In [17]:
IS1_X[:2]

,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,Distance,StrandBias
0,18,12,692,49,71,0,26.0,CCA,5.510638,0.534653,NaN,20576,0.428571
1,44,26,2060,80,79,0,33.5,ACG,2.141935,0.599010,intergenic,18084,0.612500


In [18]:
IS1_Y[:2]

,X2318876,X2319000,X2321884,X2322311,X2323876,X2340887,X2341283,X2341286,X2341288,X2341290,X2341292,X2341294,X2341781,X2342416,X2342420,X2342947,X2342949,X2343041,X2343043,X2343045,
0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...


In [19]:
totalSample = IS1_Y.shape[0]
IS1_Y_Val = np.zeros((totalSample,1)) #Will be our label now

print totalSample
print IS1_Y_Val.shape

214541
(214541, 1)


In [20]:
IS1_Y.shape

(214541, 119)

In [21]:
IS1_Y.values[0].shape

(119,)

In [22]:
for i in range(totalSample):
    IS1_Y_Val[i] = np.argmax(np.bincount(IS1_Y.values[i]))

In [23]:
IS1_Y_Val_1 = pd.DataFrame(IS1_Y_Val)
IS1_Y_Val_1.columns = ['Label']

In [24]:
IS1_Y_Val_1[:3]

,Label
0,0
1,0
2,0


In [62]:
pd.concat([IS1.iloc[:,1:16], IS1_Y_Val_1],axis=1).head()

,POS,END_x,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,Distance,StrandBias,Label
0,10290,10291,18,12,692,49,71,0,26.0,CCA,5.510638,0.534653,NaN,20576,0.428571,0
1,12782,12783,44,26,2060,80,79,0,33.5,ACG,2.141935,0.599010,intergenic,18084,0.612500,0
2,17221,17222,54,18,2079,79,114,0,48.0,GTG,2.551724,0.603960,NaN,13645,0.493671,0
3,17727,17728,45,5,1568,55,63,0,42.0,ACT,2.290541,0.584158,NaN,13139,0.272727,0
4,18090,18091,48,3,1522,57,50,0,63.0,GCC,2.528169,0.613861,NaN,12776,0.526316,0


In [25]:
IS1_dataframe = pd.concat([IS1_X, IS1_Y_Val_1],axis=1)

In [26]:
IS1_dataframe.shape

(214541, 14)

In [27]:
IS1_dataframe[:2]

,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,Distance,StrandBias,Label
0,18,12,692,49,71,0,26.0,CCA,5.510638,0.534653,NaN,20576,0.428571,0
1,44,26,2060,80,79,0,33.5,ACG,2.141935,0.599010,intergenic,18084,0.612500,0


In [51]:
IS1_dataframe.to_csv('IS1.csv')

In [52]:
#Now we have variable importance ...lets do analysis

In [53]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [54]:
%%R
# Read in the data
IS1Frame = read.csv("IS1.csv")
str(IS1Frame)

'data.frame':	214541 obs. of  15 variables:
 $ X              : int  0 1 2 3 4 5 6 7 8 9 ...
 $ RefAllele      : int  18 44 54 45 48 41 70 36 13 32 ...
 $ NonRefAllele   : int  12 26 18 5 3 5 8 4 3 8 ...
 $ BaseQual       : int  692 2060 2079 1568 1522 1460 2315 1144 459 1285 ...
 $ TumourCoverage : int  49 80 79 55 57 52 107 44 21 61 ...
 $ NormalCoverage : int  71 79 114 63 50 43 94 47 13 59 ...
 $ MapQual        : num  0 0 0 0 0 0 0 0 0 0 ...
 $ ReadPosition   : num  26 33.5 48 42 63 45.5 42 68.5 41 52 ...
 $ Trinucleotide  : Factor w/ 32 levels "ACA","ACC","ACG",..: 9 3 23 4 18 12 27 3 1 4 ...
 $ HomopolymerRate: num  5.51 2.14 2.55 2.29 2.53 ...
 $ GCcontent      : num  0.535 0.599 0.604 0.584 0.614 ...
 $ GenomicLocation: Factor w/ 16 levels "","downstream",..: 1 5 1 1 1 8 8 8 8 5 ...
 $ Distance       : int  20576 18084 13645 13139 12776 11479 10632 9643 2984 295 ...
 $ StrandBias     : num  0.429 0.613 0.494 0.273 0.526 ...
 $ Label          : num  0 0 0 0 0 0 0 0 0 0 ...


In [32]:
%%R
#sum(is.na(CPCG0183Frame$StrandBias))
#sum(is.na(IS1Frame$ReadPosition)) 
sum(is.na(IS1Frame)) 

[1] 0


In [33]:
#%%R
#CPCG0235Frame$ReadPosition[is.na(CPCG0235Frame$ReadPosition)] = mean(CPCG0235Frame$ReadPosition, na.rm=TRUE)
#CPCG0235Frame$StrandBias[is.na(CPCG0235Frame$StrandBias)] = mean(CPCG0235Frame$StrandBias, na.rm=TRUE)

#print (sum(is.na(CPCG0235Frame$ReadPosition)) )
#print (sum(is.na(CPCG0235Frame$StrandBias)) )

In [34]:
%%R
sum(is.na(IS1Frame))  #No NA value now

[1] 0


In [35]:
%%R
write.csv(IS1Frame, file = "IS1_imputed.csv")  #Cleaned CSV without NA

In [36]:
%%R
library(h2o)
localH2O = h2o.init(ip = "localhost", port = 54321, startH2O = TRUE,min_mem_size = "3g")

/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: statmod

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: 
----------------------------------------------------------------------

Your next step is to start H2O:
    > h2o.init()

For H2O package documentation, ask for help:
    > ??h2o

After starting H2O, you can use the Web UI at http://localhost:54321
For more information visit http://docs.h2o.ai

----------------------------------------------------------------------


  res = super(Function, self).__call__(*new_args, **new_kwargs)
/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: 
Attaching package: ‘h2o’


  res = super(Function, self).__call__(*new_args, **new_kwargs)
/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: The following objects are masked from ‘


H2O is not running yet, starting it now...

Note:  In case of errors look at the following log files:
    /tmp/RtmpiQpC6Z/h2o_rbhat_started_from_r.out
    /tmp/RtmpiQpC6Z/h2o_rbhat_started_from_r.err


Starting H2O JVM and connecting: .. Connection successful!

R is connected to the H2O cluster: 
    H2O cluster uptime:         1 seconds 743 milliseconds 
    H2O cluster version:        3.9.1.3435 
    H2O cluster name:           H2O_started_from_R_rbhat_xqb196 
    H2O cluster total nodes:    1 
    H2O cluster total memory:   26.60 GB 
    H2O cluster total cores:    56 
    H2O cluster allowed cores:  2 
    H2O cluster healthy:        TRUE 
    H2O Connection ip:          localhost 
    H2O Connection port:        54321 
    H2O Connection proxy:       NA 
    R Version:                  R version 3.3.0 beta (2016-03-30 r70404) 

Note:  As started, H2O is limited to the CRAN default of 2 CPUs.
       Shut down and restart H2O as shown below to use all your CPUs.
           > h2o.s

In [37]:
%%R
## Import CSV as H2O:
csvPath = "/home/rbhat/Workspace/S3Lab_Projects/SMC-MetaChallange/IS1_imputed.csv"
IS1.hex          = h2o.importFile(path = csvPath, destination_frame = "IS1.hex") 
train                 = as.data.frame(IS1.hex)

str(train)

  |======================================================================| 100%
'data.frame':	214541 obs. of  16 variables:
 $ C1             : int  1 2 3 4 5 6 7 8 9 10 ...
 $ X              : int  0 1 2 3 4 5 6 7 8 9 ...
 $ RefAllele      : int  18 44 54 45 48 41 70 36 13 32 ...
 $ NonRefAllele   : int  12 26 18 5 3 5 8 4 3 8 ...
 $ BaseQual       : int  692 2060 2079 1568 1522 1460 2315 1144 459 1285 ...
 $ TumourCoverage : int  49 80 79 55 57 52 107 44 21 61 ...
 $ NormalCoverage : int  71 79 114 63 50 43 94 47 13 59 ...
 $ MapQual        : num  0 0 0 0 0 0 0 0 0 0 ...
 $ ReadPosition   : num  26 33.5 48 42 63 45.5 42 68.5 41 52 ...
 $ Trinucleotide  : Factor w/ 32 levels "ACA","ACC","ACG",..: 9 3 23 4 18 12 27 3 1 4 ...
 $ HomopolymerRate: num  5.51 2.14 2.55 2.29 2.53 ...
 $ GCcontent      : num  0.535 0.599 0.604 0.584 0.614 ...
 $ GenomicLocation: Factor w/ 15 levels "downstream","exonic",..: NA 4 NA NA NA 7 7 7 7 4 ...
 $ Distance       : int  20576 18084 13645 13139 12776 114

In [38]:
%%R

train$Trinucleotide   = as.factor(train$Trinucleotide)
train$GenomicLocation = as.factor(train$GenomicLocation)
train$Label           = as.factor(train$Label)

train_h2o             = as.h2o(train)

  |======================================================================| 100%


In [39]:
%%R
#Training a deep learning model:---------------------------------------------------------#
model <- h2o.deeplearning( x = 3:15, # column numbers for predictors
                           y = 16,            # column number for label
                           training_frame = train_h2o,
                           activation = "RectifierWithDropout", 
                           input_dropout_ratio = 0.2,
                           hidden_dropout_ratios = c(0.5,0.5),
                           balance_classes = TRUE, 
                           hidden = c(10,8),  ## two hidden layers
                           epochs = 50,
                           variable_importances = T)

  |======================================================================| 100%


In [40]:
%%R
var_importance = h2o.varimp(model)
#print(var_importance)
h2o.varimp(model)

Variable Importances: 
                     variable relative_importance scaled_importance percentage
1                NonRefAllele            1.000000          1.000000   0.347961
2                   RefAllele            0.306104          0.306104   0.106512
3                    BaseQual            0.204257          0.204257   0.071073
4              NormalCoverage            0.118062          0.118062   0.041081
5 GenomicLocation.missing(NA)            0.091290          0.091290   0.031765

---
                     variable relative_importance scaled_importance percentage
55          Trinucleotide.CTA            0.006931          0.006931   0.002412
56          Trinucleotide.ACT            0.006452          0.006452   0.002245
57          Trinucleotide.GTC            0.006219          0.006219   0.002164
58   GenomicLocation.intronic            0.006110          0.006110   0.002126
59 GenomicLocation.intergenic            0.001992          0.001992   0.000693
60  Trinucleotide.missin

In [41]:
%%R
write.table(var_importance, file="IS1_variable_importance.csv")

In [42]:
var_importance = pd.read_csv(open('IS1_variable_importance.csv','r'),sep=' ')
#pd.read_csv('CPCG0100_variable_importance.csv')

In [43]:
var_importance[:5]

,variable,relative_importance,scaled_importance,percentage
1,NonRefAllele,1.000000,1.000000,0.347961
2,RefAllele,0.306104,0.306104,0.106512
3,BaseQual,0.204257,0.204257,0.071073
4,NormalCoverage,0.118062,0.118062,0.041081
5,GenomicLocation.missing(NA),0.091290,0.091290,0.031765


In [44]:
GenomicFeatures_IS1[:5]

,CHROM,POS,END,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,Distance,StrandBias
0,1,10290,10291,18,12,692,49,71,0,26.0,CCA,5.510638,0.534653,NaN,20576,0.428571
1,1,12782,12783,44,26,2060,80,79,0,33.5,ACG,2.141935,0.599010,intergenic,18084,0.612500
2,1,17221,17222,54,18,2079,79,114,0,48.0,GTG,2.551724,0.603960,NaN,13645,0.493671
3,1,17727,17728,45,5,1568,55,63,0,42.0,ACT,2.290541,0.584158,NaN,13139,0.272727
4,1,18090,18091,48,3,1522,57,50,0,63.0,GCC,2.528169,0.613861,NaN,12776,0.526316


In [45]:
var_imp_dict = {}

for i in range(var_importance.shape[0]):
    var_imp_dict[var_importance.iloc[i,0]] = var_importance.iloc[i,1]

var_imp_dict

{'BaseQual': 0.204256847500801,
 'Distance': 0.034344896674156203,
 'GCcontent': 0.0095686884596943907,
 'GenomicLocation.UTR3': 0.010045447386801199,
 'GenomicLocation.UTR5': 0.0231131184846163,
 'GenomicLocation.downstream': 0.010389370843768101,
 'GenomicLocation.exonic': 0.010724452324211599,
 'GenomicLocation.exonic;splicing': 0.0379977077245712,
 'GenomicLocation.intergenic': 0.00199206522665918,
 'GenomicLocation.intronic': 0.00611010799184442,
 'GenomicLocation.missing(NA)': 0.091290399432182298,
 'GenomicLocation.ncRNA_UTR3': 0.0130932005122304,
 'GenomicLocation.ncRNA_UTR5': 0.0297621358186007,
 'GenomicLocation.ncRNA_exonic': 0.023482041433453602,
 'GenomicLocation.ncRNA_intronic': 0.0225302185863256,
 'GenomicLocation.ncRNA_splicing': 0.027009967714548101,
 'GenomicLocation.splicing': 0.024021008983254401,
 'GenomicLocation.upstream': 0.014306032098829699,
 'GenomicLocation.upstream;downstream': 0.021003548055887201,
 'HomopolymerRate': 0.030462052673101401,
 'MapQual': 0.0

In [46]:
GenomicFeatures_IS1.columns

Index([u'CHROM', u'POS', u'END', u'RefAllele', u'NonRefAllele', u'BaseQual', u'TumourCoverage', u'NormalCoverage', u'MapQual', u'ReadPosition', u'Trinucleotide', u'HomopolymerRate', u'GCcontent', u'GenomicLocation', u'Distance', u'StrandBias'], dtype='object')

In [47]:
GenomicFeatures_IS1.dtypes

CHROM               object
POS                float64
END                  int64
RefAllele            int64
NonRefAllele         int64
BaseQual             int64
TumourCoverage       int64
NormalCoverage       int64
MapQual            float64
ReadPosition       float64
Trinucleotide       object
HomopolymerRate    float64
GCcontent          float64
GenomicLocation     object
Distance             int64
StrandBias         float64
dtype: object

In [48]:
GenomicFeatures_IS1.iloc[0,13]

nan

In [49]:
GenomicFeatures_IS1[:2]
#print GenomicFeatures_IS1.iloc[0,2]

,CHROM,POS,END,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,Distance,StrandBias
0,1,10290,10291,18,12,692,49,71,0,26.0,CCA,5.510638,0.534653,NaN,20576,0.428571
1,1,12782,12783,44,26,2060,80,79,0,33.5,ACG,2.141935,0.599010,intergenic,18084,0.612500


In [50]:
noOfSample = GenomicFeatures_IS1.shape[0]

IS1_Dict = {}

for i in range(noOfSample):
    
    pos = GenomicFeatures_IS1.iloc[i,1] #gets the pos value
    
    RefAllele      = GenomicFeatures_IS1.iloc[i,3]*var_imp_dict['RefAllele']
    NonRefAllele   = GenomicFeatures_IS1.iloc[i,4]*var_imp_dict['NonRefAllele']
    BaseQual       = GenomicFeatures_IS1.iloc[i,5]*var_imp_dict['BaseQual']
    TumourCoverage = GenomicFeatures_IS1.iloc[i,6]*var_imp_dict['TumourCoverage']
    NormalCoverage = GenomicFeatures_IS1.iloc[i,7]*var_imp_dict['NormalCoverage']
    MapQual        = GenomicFeatures_IS1.iloc[i,8]*var_imp_dict['MapQual']
    ReadPosition   = GenomicFeatures_IS1.iloc[i,9]*var_imp_dict['ReadPosition']
    
    Trinucleotide  = 0
    if isinstance(GenomicFeatures_IS1.iloc[i,10], str):
        Trinucleotide  = var_imp_dict['Trinucleotide.'+str(GenomicFeatures_IS1.iloc[i,10])]
    HomopolymerRate= GenomicFeatures_IS1.iloc[i,11]*var_imp_dict['HomopolymerRate']
    GCcontent      = GenomicFeatures_IS1.iloc[i,12]*var_imp_dict['GCcontent']
    
    GenomicLocation = 0
    if isinstance(GenomicFeatures_IS1.iloc[i,13], str):
        GenomicLocation= var_imp_dict['GenomicLocation.'+str(GenomicFeatures_IS1.iloc[i,13])]
    Distance    = GenomicFeatures_IS1.iloc[i,14]*var_imp_dict['Distance']
    StrandBias     = GenomicFeatures_IS1.iloc[i,15]*var_imp_dict['StrandBias']
    
    #print 'RefAllele=',RefAllele,' NonRefAllele=',NonRefAllele,' BaseQual=',BaseQual,' TumourCoverage=',TumourCoverage
    #print 'NormalCoverage=',NormalCoverage,' MapQual=',MapQual,' ReadPosition=',ReadPosition,' Trinucleotide=',Trinucleotide
    #print 'HomopolymerRate=',HomopolymerRate,' GCcontent=',GCcontent,' GenomicLocation=',GenomicLocation,' Distance=',Distance
    #print 'StrandBias=',StrandBias
    IS1_Dict[pos] = RefAllele+NonRefAllele+BaseQual+TumourCoverage+NormalCoverage+MapQual+ReadPosition+Trinucleotide+ \
                         HomopolymerRate+GCcontent+GenomicLocation+Distance+StrandBias

In [55]:
#Sorting dictionary
IS1_Dict1 = [(k,v) for v,k in sorted([(v,k) for k,v in IS1_Dict.items()],reverse=True)]

In [56]:
IS1_Dict_Frame = pd.DataFrame(IS1_Dict1, columns=['POS', 'Value'])

In [57]:
IS1_Dict_Frame.to_csv("IS1_Dict_Frame.csv")

In [58]:
IS1_Dict_Frame[:10]

,POS,Value
0,121485149,173121.760915
1,121485158,169391.078963
2,121485137,168720.305531
3,121485185,164629.580086
4,121485360,144549.036753
5,42385225,125146.527530
6,42385232,124388.951764
7,121485077,121639.249308
8,42385213,121418.224106
9,42385203,118002.132140


In [65]:
GenomicFeatures_IS1_1 = pd.concat([IS1.iloc[:,0:16], IS1_Y_Val_1],axis=1)

In [66]:
rows = IS1_Dict_Frame['POS'][:20].values
print rows
GenomicFeatures_IS1_Top20 = GenomicFeatures_IS1_1.loc[GenomicFeatures_IS1_1['POS'].isin(rows)]

[  1.21485149e+08   1.21485158e+08   1.21485137e+08   1.21485185e+08
   1.21485360e+08   4.23852250e+07   4.23852320e+07   1.21485077e+08
   4.23852130e+07   4.23852030e+07   2.44908430e+07   2.44882870e+07
   2.52933280e+07   2.52933540e+07   1.21485373e+08   4.23852770e+07
   4.25996320e+07   1.21484917e+08   1.21484919e+08   1.21485045e+08]


In [69]:
GenomicFeatures_IS1_Top20[:10]

,CHROM,POS,END_x,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,Distance,StrandBias,Label
8686,1,121484917,121484918,7773,71,269063,206217,206365,29,43,TCT,2.493056,0.376238,NaN,8682,0.646285,0
8687,1,121484919,121484920,7752,76,269485,203857,203950,29,44,ATT,2.493056,0.371287,NaN,8684,0.646630,0
8691,1,121485045,121485046,7933,63,261516,211092,210724,29,40,GCC,2.065789,0.400990,NaN,8810,0.722979,0
8692,1,121485077,121485078,7818,145,267436,306953,306674,29,31,GCA,1.967320,0.405941,NaN,8842,0.695898,0
8693,1,121485137,121485138,7930,65,277189,522907,522318,29,40,TTC,2.406897,0.405941,NaN,8902,0.535900,0
8694,1,121485149,121485150,7874,102,266679,554323,553369,29,42,ACA,2.335616,0.396040,NaN,8914,0.499195,0
8695,1,121485158,121485159,7883,78,276317,526509,526734,29,44,TCT,2.308725,0.391089,NaN,8923,0.483494,0
8696,1,121485185,121485186,7780,86,270374,509762,509798,29,55,TTG,2.288591,0.381188,NaN,8950,0.463304,0
8697,1,121485360,121485361,7965,106,282221,402770,401072,29,60,CTT,7.937984,0.351485,NaN,9125,0.327969,0
8698,1,121485373,121485374,5397,41,176499,340599,339309,29,65,GTC,15.747899,0.331683,NaN,9138,0.295832,0


In [61]:
GenomicFeatures_IS1_Top20.to_csv('GenomicFeatures_IS1_Top20.csv')

In [64]:
pd.concat([IS1.iloc[:,0:16], IS1_Y_Val_1],axis=1).head()

,CHROM,POS,END_x,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,Distance,StrandBias,Label
0,1,10290,10291,18,12,692,49,71,0,26.0,CCA,5.510638,0.534653,NaN,20576,0.428571,0
1,1,12782,12783,44,26,2060,80,79,0,33.5,ACG,2.141935,0.599010,intergenic,18084,0.612500,0
2,1,17221,17222,54,18,2079,79,114,0,48.0,GTG,2.551724,0.603960,NaN,13645,0.493671,0
3,1,17727,17728,45,5,1568,55,63,0,42.0,ACT,2.290541,0.584158,NaN,13139,0.272727,0
4,1,18090,18091,48,3,1522,57,50,0,63.0,GCC,2.528169,0.613861,NaN,12776,0.526316,0
